In [ ]:
# Google Collab Requirements / Checks
from google.colab.patches import cv2_imshow # Change this before running elsewhere!!

from google.colab import drive
drive.mount('/content/gdrive')

# Imports
import cv2
import os
import glob
import math
import numpy as np
import pandas as pd
import copy

from PIL import Image
from torchvision import transforms
from shapely.geometry import Polygon

Mounted at /content/gdrive


In [ ]:
# Load manual ms coco dataset and bounding boxes
# Varun's Directory
#dir_uri = '/content/gdrive/MyDrive/University of Wisconsin-Madison/2021 Fall/cs762'
# Devesh's Directory
dir_uri = '/content/gdrive/MyDrive'
proj_dir = dir_uri + '/CS762_Deep_Learning_Project'

# Images
dataset_folder = dir_uri + '/CS762_Deep_Learning_Project/datasets/Manual_MSCOCO' 
image_names = []
for filename in os.listdir(dataset_folder):
  image_names.append(filename)

image_names.sort(key=lambda x: int(x.split('-')[0]))

# Load bbox's
bbox_path = proj_dir + '/fullBBOX.xlsx'
bbox_df = pd.read_excel(bbox_path)
all_boxes = bbox_df.to_numpy()

detr_bbox_path = proj_dir + '/detr_bbox.csv'
detr_bbox_df = pd.read_csv(detr_bbox_path)
detr_boxes = detr_bbox_df.to_numpy()

yolo_bbox_path = proj_dir + '/yolo_bbox.csv'
yolo_bbox_df = pd.read_csv(yolo_bbox_path)
yolo_boxes = yolo_bbox_df.to_numpy()

deit_bbox_path = proj_dir + '/deit_bbox.csv'
deit_bbox_df = pd.read_csv(deit_bbox_path)
deit_boxes = deit_bbox_df.to_numpy()

resnet_bbox_path = proj_dir + '/resnet_bbox.csv'
resnet_bbox_df = pd.read_csv(resnet_bbox_path)
resnet_boxes = resnet_bbox_df.to_numpy()

output_dir = proj_dir + '/Q2/output_images_bbox_drawn'

In [ ]:
def resizeBbox(orig_bbox, orig_img_size, new_img_size):
    orig_h = orig_img_size[0]
    orig_w = orig_img_size[1]
    new_h = new_img_size[0]
    new_w = new_img_size[1]

    h_ratio = new_h / orig_h
    w_ratio = new_w / orig_w

    # new_bbox = copy.deepcopy(orig_bbox)
    # new_bbox[2] = orig_bbox[2] * w_ratio
    # new_bbox[3] = orig_bbox[3] * h_ratio
    # new_bbox[4] = orig_bbox[4] * w_ratio
    # new_bbox[5] = orig_bbox[5] * h_ratio
    # new_bbox[6] = orig_bbox[6] * w_ratio
    # new_bbox[7] = orig_bbox[7] * h_ratio
    # new_bbox[8] = orig_bbox[8] * w_ratio
    # new_bbox[9] = orig_bbox[9] * h_ratio

    new_bbox = [
        orig_bbox[0] * w_ratio,
        orig_bbox[1] * h_ratio,

        orig_bbox[2] * w_ratio,
        orig_bbox[3] * h_ratio,

        orig_bbox[4] * w_ratio,
        orig_bbox[5] * h_ratio,

        orig_bbox[6] * w_ratio,
        orig_bbox[7] * h_ratio
    ]

    return new_bbox

def get_box( name, boxes ):
  for box in boxes:
    if box[0] == name:
      return box

  return None

In [ ]:
detr_transform = transforms.Compose([
    transforms.Resize(800),
])

yolo_image_shape = ( 416, 416, 3 )
# resnet_image_shape = ( 224, 224, 3 )
# deit_image_shape = ( 224, 224, 3 )
classifier_image_shape = ( 224, 224, 3 )

In [ ]:
boxes_to_analyze = {
    'detr': ( detr_boxes, 'te' ),
    'yolo': ( yolo_boxes, 'te' ),
    # 'deit': ( deit_boxes, 'te' ),
    # 'resnet': ( resnet_boxes, 'te' )
}

images_to_vislualize = [ 53]
for box_type, box_data in boxes_to_analyze.items():
  boxes = box_data[0]

  model = box_type
  data = []

  for i in images_to_vislualize:

    # Get boxes
    box_data = boxes[i]
    image_num = box_data[0]

    # Print
    print( 'Running for model {} on box {}, image num {}, original image name {}'.format( model, i, image_num, image_names[image_num] ) )

    image2 = Image.open( dataset_folder + '/' + image_names[image_num] ).convert('RGB')
    image = cv2.imread( dataset_folder + '/' + image_names[image_num] )
    orig_img_shape = np.array( image ).shape

    new_image = None
    new_img_shape = None
    if model == 'detr':
      new_image = detr_transform( image2 )
      new_img_shape = np.array( new_image ).shape
    elif model == 'yolo':
      new_img_shape = yolo_image_shape
    else:
      new_img_shape = classifier_image_shape
    
    orig_img_shape = ( orig_img_shape[0], orig_img_shape[1] )
    new_img_shape = ( new_img_shape[0], new_img_shape[1] )

    orig_box = get_box( image_num, all_boxes )[2:10]
    # orig_box = resizeBbox( orig_box, orig_img_shape, new_img_shape )
    
    gradcam_box = box_data[2:10]
    gradcam_box = resizeBbox( gradcam_box, new_img_shape, orig_img_shape )

    # print(orig_box)
    # print(gradcam_box)

    start_point_gt = ( int( orig_box[0] ), int( orig_box[1] ) )
    end_point_gt = ( int( orig_box[4] ), int( orig_box[5] ) )
    bbox_gt = cv2.rectangle( image, start_point_gt, end_point_gt, (0, 0, 255), 2 )

    start_point_grad = ( int( gradcam_box[0] ), int( gradcam_box[1] ) )
    end_point_grad = ( int( gradcam_box[4] ), int( gradcam_box[5] ) )
    bbox_gradcam = cv2.rectangle( image, start_point_grad, end_point_grad, (0, 255, 0), 2 )

    image_name = image_names[image_num].split('.')[0]
    cv2.imwrite( output_dir + '/' + image_name + '_' + model + '.jpeg', image )

Running for model detr on box 53, image num 53, original image name 53-zebra.jpg
Running for model yolo on box 53, image num 53, original image name 53-zebra.jpg
